# Vector #1: Vector2d Compatible
Making  Vector to  be a standalone example of a class implementing the sequence protocol.

In [1]:
from array import array
import reprlib
import math

In [2]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # protected

    def __iter__(self):
        return iter(self._components)  # to allow iteration

    def __repr__(self):
        components = reprlib.repr(self._components)  # to  get limited-length representation
        components = components[components.find('['):-1]  # remove 'd' prefix and trailing before using as constructor
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # build a bytes object directly

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # can't use hypot anymore

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # instead of frombytes, we pass the memview directly without unpacking

In [3]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [4]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [6]:
v1 = Vector([3, 4])
x, y = v1
x, y

(3.0, 4.0)

In [7]:
v1

Vector([3.0, 4.0])

In [8]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [9]:
print(v1)

(3.0, 4.0)


In [10]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [11]:
abs(v1)

5.0

In [12]:
bool(v1), bool(Vector([0, 0]))

(True, False)

In [13]:
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

Vector([3.0, 4.0])

In [14]:
v1 == v1_clone

True

## Tests with 3-dimensions:

In [15]:
v1 = Vector([3, 4, 5])
x, y, z = v1
x, y, z

(3.0, 4.0, 5.0)

In [16]:
v1

Vector([3.0, 4.0, 5.0])

In [17]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [18]:
print(v1)

(3.0, 4.0, 5.0)


In [19]:
abs(v1)

7.0710678118654755

In [20]:
bool(v1), bool(Vector([0, 0, 0]))

(True, False)

## Tests with many dimensions:

In [21]:
v7 = Vector(range(7))
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [22]:
abs(v7)

9.539392014169456

In [23]:
v1 = Vector([3, 4, 5])
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

Vector([3.0, 4.0, 5.0])

In [24]:
v1 == v1_clone

True

# Protocols and Duck Typing
## Protocol
Is an informal interface, defined only in documentations, and not in code.
### Sequence Protocol Implementation Example :

In [25]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'Spades Diamonds Clubs Hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits
                                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

## Duck Typing
We say the above is a sequence because it behaves like a sequence.
# Vector #2: A Slicable Sequence

In [26]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR_V2
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]
# END VECTOR_V2

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [27]:
v7 = Vector(range(7))
len(v7)

7

In [28]:
v7[0], v7[-1]

(0.0, 6.0)

In [29]:
v7[1:4]

array('d', [1.0, 2.0, 3.0])

## How slicking works

In [30]:
class MySeq:
    def __getitem__(self, index):
        return index

In [31]:
s = MySeq()
s[1]

1

In [32]:
s[1:4]

slice(1, 4, None)

In [33]:
# start at 1, stop at 4, step by 2
s[1:4:2]

slice(1, 4, 2)

In [34]:
# __getitem__ receives a tuple
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [35]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [36]:
slice  # a built-in type

slice

In [37]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

###  S.indices(len) -> (start, stop, stride)

In [38]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [39]:
slice(-3, None, None).indices(5)

(2, 5, 1)

## A Slice-Aware __getitem__

In [40]:
import numbers

In [41]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR_V2
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)  # get the class of the instance
        if isinstance(index, slice):  # the index argument is a slice
            return cls(self._components[index])  # invoke the class to buildd another Vector instance
        elif isinstance(index, numbers.Integral):  # Check if the index is an int
            return self._components[index] 
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))  
# END VECTOR_V2

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [42]:
v7 = Vector(range(7))
v7[-1]

6.0

In [43]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [44]:
v7[-1:]

Vector([6.0])

In [45]:
v7[1,2]

TypeError: Vector indices must be integers

# Vector #3: Dynamic Attribute Access
Access by name, e.g. x,y,z

The `__getattr__` method is invoked when attribute lookup fails.

In [46]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

# BEGIN VECTOR_V3_GETATTR
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:  # if the name is single character
            pos = cls.shortcut_names.find(name)  # find position
            if 0 <= pos < len(self._components):  # return element if within range
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
# END VECTOR_V3_GETATTR

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [47]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [48]:
v.x

0.0

In [49]:
v.x = 10

In [50]:
v.x

10

In [51]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [52]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

# BEGIN VECTOR_V3_GETATTR
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  
        if len(name) == 1:  
            pos = cls.shortcut_names.find(name)  
            if 0 <= pos < len(self._components): 
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
# END VECTOR_V3_GETATTR

# BEGIN VECTOR_V3_SETATTR
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # special  handdling for single-character attribute names
            if name in cls.shortcut_names:  # set specific error message
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # if lower case, set error
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # otherwise blank
            if error:  # if non blank 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # call superclass for standard behaviour

# END VECTOR_V3_SETATTR

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [53]:
v = Vector(range(5))
v.x = 10

AttributeError: readonly attribute 'x'

# Vector #4: Hashing and Faster `==`
## `reduce`
<img src="reduce.png" width="75%">

In [54]:
import functools
import operator

In [55]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        hashes = (hash(x) for x in self._components) # create a generator expreession
        return functools.reduce(operator.xor, hashes, 0) # feed hashes to reduce with ^ function

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

## map reduce
<img src="mapreduce.png" width="75%">
The mapping step produces one has for each component, and the reduce step aggregates all hashees with xor operator.

`map` is lazy: it creates a generator that yields the result on demand, thus saving memory

In [56]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other))) # zip prodduces a generator of tuples

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [57]:
v1 = Vector([3, 4])
v2 = Vector([3.1, 4.2])
v3 = Vector([3, 4, 5])
v6 = Vector(range(6))
hash(v1), hash(v3), hash(v6)

(7, 2, 1)

In [58]:
# Most hash values of non-integers vary from a 32-bit to 64-bit CPython build:

import sys

hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)

True

# Vector #3: Formatting

In [59]:
import itertools  # to use `chain` function

In [60]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):  # compute one of the angular coordinates
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # create genexp to compute all angles
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # produce genexp to iterate over magnitude and angular coord.
            outer_fmt = '<{}>'  # configure spherical coorddinate display
        else:
            coords = self
            outer_fmt = '({})'  # configure Cartesian coordinate display
        components = (format(c, fmt_spec) for c in coords)  # create genexp to format each coordinate item
        return outer_fmt.format(', '.join(components))  # plug formatted components

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [61]:
format(v1)

'(3.0, 4.0)'

In [62]:
format(v1, '.2f')

TypeError: unsupported format string passed to Vector.__format__

In [63]:
format(v1, '.3e')

TypeError: unsupported format string passed to Vector.__format__

In [64]:
format(v3)

'(3.0, 4.0, 5.0)'

In [65]:
format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

### Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D

In [66]:
format(Vector([1, 1]), 'h')

'<1.4142135623730951, 0.7853981633974483>'

In [67]:
format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [68]:
format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [69]:
format(Vector([1, 1, 1]), 'h')

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [70]:
format(Vector([2, 2, 2]), '.3eh')

'<3.464e+00, 9.553e-01, 7.854e-01>'

In [71]:
format(Vector([0, 0, 0]), '0.5fh')

'<0.00000, 0.00000, 0.00000>'

In [72]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [73]:
format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [74]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'